# Test Kubeflow Trainer Integration

This example notebook is loosely based on the following upstream examples:
* [TrainJob for PyTorch with default runtime](https://github.com/kubeflow/trainer/blob/v2.0.1/test/e2e/e2e_test.go)
* [TrainJob for PyTorch with custom training function](https://github.com/kubeflow/trainer/blob/v2.0.1/examples/pytorch/image-classification/mnist.ipynb)

Note that the above can get out of sync with the actual testing upstream does, so make sure to also check out [upstream examples](https://github.com/kubeflow/trainer/blob/v2.0.1/examples/) and [E2E tests](https://github.com/kubeflow/trainer/blob/v2.0.1/test/) for updating the notebook.

The workflow is:
- create training job
- monitor its execution
- get training logs
- delete job

## Setup

In [ ]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

### Import required packages

In [ ]:
from kubeflow.trainer import CustomTrainer, TrainerClient
from tenacity import retry, stop_after_attempt, wait_exponential

### Initialise Training Client

We will be using the Training SDK for any actions executed as part of this example.

In [ ]:
client = TrainerClient()

### Define Helper to print training logs

In [ ]:
def print_training_logs(client, job_name: str):
    for logline in client.get_job_logs(job_name, follow=True):
        print(logline)

### Define Helper to assert TrainJob removal

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_job_removed(client, job_name):
    """Wait for TrainJob to be removed."""
    # fetch the existing TrainJob names
    # verify that the Job was deleted successfully
    jobs = {job.metadata.name for job in client.list_jobs()}
    assert job_name not in jobs, f"Failed to delete TrainJob {job_name}!"

### List the Training Runtimes
You can get the list of available Training Runtimes to start your TrainJob.

Additionally, it might show available accelerator type and number of available resources.

In [ ]:
for runtime in client.list_runtimes():
    print(runtime)
    if runtime.name == "torch-distributed":
        torch_runtime = runtime

## Test TrainJob with default ClusterTraingRuntime template


In [ ]:
job_name = client.train(
    trainer=CustomTrainer(
        # Set how many PyTorch nodes you want to use for distributed training.
        num_nodes=2,
        # Set the resources for each PyTorch node.
        resources_per_node={
            "cpu": 3,
            "memory": "4Gi",
            # Uncomment this to distribute the TrainJob using GPU nodes.
            # "nvidia.com/gpu": 1,
        },
    ),
    runtime=torch_runtime,
)

### Check the TrainJob steps
You can check the components of TrainJob that's created.

Since the TrainJob performs distributed training across 2 nodes, it generates 2 steps: trainer-node-0 .. trainer-node-1.

You can get the individual status for each of these steps.

In [ ]:
client.wait_for_job_status(name=job_name, status={"Running"})

In [ ]:
for c in client.get_job(name=job_name).steps:
    print(f"Step: {c.name}, Status: {c.status}, Devices: {c.device} x {c.device_count}\n")

### Get TrainJob logs
Get and print the training logs of the TrainJob with the training steps.

In [ ]:
print_training_logs(client, job_name)

### Delete TrainJob

Delete the created TrainJob and check it is removed.

In [ ]:
client.delete_job(name=job_name)

In [ ]:
# wait for TrainJob resources to be removed successfully
assert_job_removed(client, job_name)

## Test TrainJob with custom training function
You can use `TrainerClient()` from the Kubeflow SDK to communicate with Kubeflow Trainer APIs and scale your training function across multiple PyTorch training nodes.

`TrainerClient()` verifies that you have required access to the Kubernetes cluster.

Kubeflow Trainer creates a `TrainJob` resource and automatically sets the appropriate environment variables to set up PyTorch in distributed environment.

### Define the training function

The first step is to create function to train CNN model using Fashion MNIST data.

In [ ]:
def train_fashion_mnist():
    import os

    import torch
    import torch.distributed as dist
    import torch.nn.functional as F
    from torch import nn
    from torch.utils.data import DataLoader, DistributedSampler
    from torchvision import datasets, transforms

    # Define the PyTorch CNN model to be trained
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    device, backend = ("cpu", "gloo")
    print(f"Using Device: {device}, Backend: {backend}")

    # Setup PyTorch distributed.
    local_rank = int(os.getenv("LOCAL_RANK", 0))
    dist.init_process_group(backend=backend)
    print(
        "Distributed Training for WORLD_SIZE: {}, RANK: {}, LOCAL_RANK: {}".format(
            dist.get_world_size(),
            dist.get_rank(),
            local_rank,
        )
    )

    # Create the model and load it into the device.
    device = torch.device(f"{device}:{local_rank}")
    model = nn.parallel.DistributedDataParallel(Net().to(device))
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    # Download FashionMNIST dataset only on local_rank=0 process.
    if local_rank == 0:
        dataset = datasets.FashionMNIST(
            "./data",
            train=True,
            download=True,
            transform=transforms.Compose([transforms.ToTensor()]),
        )
    dist.barrier()
    dataset = datasets.FashionMNIST(
        "./data",
        train=True,
        download=False,
        transform=transforms.Compose([transforms.ToTensor()]),
    )

    # Shard the dataset across workers.
    train_loader = DataLoader(dataset, batch_size=100, sampler=DistributedSampler(dataset))

    # TODO(astefanutti): add parameters to the training function
    dist.barrier()
    for epoch in range(1, 3):
        model.train()

        # Iterate over mini-batches from the training set
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Copy the data to the GPU device if available
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(inputs)
            loss = F.nll_loss(outputs, labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0 and dist.get_rank() == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(inputs),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    # Wait for the distributed training to complete
    dist.barrier()
    if dist.get_rank() == 0:
        print("Training is finished")

    # Finally clean up PyTorch distributed
    dist.destroy_process_group()

### Run the Distributed TrainJob
Kubeflow TrainJob will train the above model on 2 PyTorch nodes.

In [ ]:
job_name = client.train(
    trainer=CustomTrainer(
        func=train_fashion_mnist,
        # Set how many PyTorch nodes you want to use for distributed training.
        num_nodes=2,
        # Set the resources for each PyTorch node.
        resources_per_node={
            "cpu": 3,
            "memory": "4Gi",
            # Uncomment this to distribute the TrainJob using GPU nodes.
            # "nvidia.com/gpu": 1,
        },
    ),
    runtime=torch_runtime,
)

### Check the TrainJob steps
You can check the components of TrainJob that's created.

Since the TrainJob performs distributed training across 2 nodes, it generates 2 steps: trainer-node-0 .. trainer-node-1.

You can get the individual status for each of these steps.

In [ ]:
client.wait_for_job_status(name=job_name, status={"Running"})

In [ ]:
for c in client.get_job(name=job_name).steps:
    print(f"Step: {c.name}, Status: {c.status}, Devices: {c.device} x {c.device_count}\n")

### Get TrainJob logs
Get and print the training logs of the TrainJob with the training steps.

In [ ]:
print_training_logs(client, job_name)

### Delete TrainJob

Delete the created TrainJob and check it is removed.

In [ ]:
client.delete_job(name=job_name)

In [ ]:
# wait for TrainJob resources to be removed successfully
assert_job_removed(client, job_name)